In [61]:
from math import radians,sin,cos,atan2,sqrt
import datetime

def getDistance(lat1,lat2,lon1,lon2):
    #=acos(sin(lat1)*sin(lat2)+cos(lat1)*cos(lat2)*cos(lon2-lon1))*6371
    #return math.acos(math.sin(lat1)*math.sin(lat2)+math.cos(lat1)*math.cos(lat2)*math.cos(lon1-lon2))*6371
    R = 6373.0
    dlon = radians(lon2 - lon1)
    dlat = radians(lat2 - lat1)
    
    a = (sin(dlat / 2) * sin(dlat / 2) +
         cos(radians(lat1)) * cos(radians(lat2)) *
         sin(dlon / 2) * sin(dlon / 2))
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    
#    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
#    c = 2 * atan2(sqrt(a), sqrt(1 - a))
#https://stackoverflow.com/questions/7477003/calculating-new-longitude-latitude-from-old-n-meters
    distance = R * c
    return distance

#https://cloud.google.com/blog/products/maps-platform/how-calculate-distances-map-maps-javascript-api

def calcNearStations(newPoints,e):
    for pointIni in newPoints:
        ListIni = newPoints[pointIni]
        for pointTar in newPoints:
            ListTar = newPoints[pointTar]
            if ListIni[2] != ListTar[2]:
                lat1 = ListIni[0]
                lat2 = ListTar[0]
                lon1 = ListIni[1]
                lon2 = ListTar[1]
                dis = getDistance(lat1,lat2,lon1,lon2)*1000 
                if  dis < e:
                    ListIni[4][pointTar] = [dis,ListTar[0],ListTar[1]]
                    ListIni[5]+=1
    writefile = open("NearSTations.txt",'w')
    for pointIni in newPoints:
        if newPoints[pointIni][5] > 0:
            strWrite = pointIni+" "+str(newPoints[pointIni][5])+" Near  "+str(newPoints[pointIni][0])+","+str(newPoints[pointIni][1])+"\n"
            writefile.write(strWrite)
            bliski = newPoints[pointIni][4]
            for stName,stDict in bliski.items():
                writefile.write("%10s %6.2f %f %f\n" %(stName,stDict[0],stDict[1],stDict[2]))
            writefile.write("\n")  
    writefile.close()     


                

    
def printFinalList(fileName,newPoints,status):
    finalList = open(fileName,'w')
    countAll = 0
    for p in newPoints:
        if status != 'ALL':
            if status == newPoints[p][3]:
                txttoWrite = "{:s} {:s} {:.10f},{:.10f}\n".format(p,newPoints[p][3],newPoints[p][0],newPoints[p][0])
                finalList.write(txttoWrite)
                countAll+=1
        else:    
            txttoWrite = "{:s} {:s} {:.10f},{:.10f}\n".format(p,newPoints[p][3],newPoints[p][0],newPoints[p][0])
            finalList.write(txttoWrite)
            countAll+=1

    txttoWrite = "Number of points={:d}\n".format(countAll)
    finalList.write(txttoWrite)
    finalList.close()
        
def CalcMedium(st1,st2):
    i = 0
    maxLat = 0
    minLat = 0
    maxLon = 0
    minLon = 0
    for s in stations:
        if i == 0:
            maxLat = stations[s][1]
            minLat = stations[s][1]
            maxLon = stations[s][2]
            minLon = stations[s][2]
        else:    
            if stations[s][1] < minLat:
                minLat = stations[s][1]
            if stations[s][1] > maxLat:
                maxLat = stations[s][1]
            if stations[s][2] < minLon:
                minLon = stations[s][2]
            if stations[s][2] > maxLon:
                maxLon = stations[s][2]
        i+=1        
    newLat = (maxLat + minLat)/2
    newLon = (maxLon + minLon)/2
    return [newLat,newLon]               


def calcSimilarStations(newPoints,e):
    similarFactor = 20
    similarity = {}
    for pb in newPoints:
        t_pb = pb
        pb = pb.split('_')
        if pb[0] not in similarity.keys():
            similarity[pb[0]] = {}
        for pe in newPoints:
            t_pe = pe
            pe = pe.split('_')
            if pb[0] != pe[0]:
                if pe[0] not in similarity[pb[0]].keys():
                    similarity[pb[0]][pe[0]] = 0 
                dis = getDistance(newPoints[t_pb][0],newPoints[t_pe][0],newPoints[t_pb][1],newPoints[t_pe][1])*1000 
                if  dis < e:
                    similarity[pb[0]][pe[0]] +=1
    simFile = open("Similarity",'w')
    for s in similarity:
        txttoWrite = "Similar with line={:s}\n".format(s)
        simFile.write(txttoWrite)   
        for sl,sv in similarity[s].items():
            txttoWrite = "({:s},{:d}) ".format(sl,sv)
            simFile.write(txttoWrite)   
        simFile.write('\n\n')
    simFile.close()   
    for s in similarity:
        for kv in similarity[s]:
            if similarity[s][kv] >= similarFactor:
                for pb in newPoints:
                       if newPoints[pb][2] == kv:
                            newPoints[pb][3] = 'Similar with:'+s
                similarity[kv][s] = 0            
                
def calcMin(newPoints,e):
    i = 0
    while True:
        if i > 0:
            pointName = lisDis[0]+'&'+lisDis[1]
            newPoints[pointName] = pointLocation
            print(lisDis)
        minDis = e
        lisDis = []
        noMore = False
        print(i)
        i+=1
        for pb in newPoints:
            for pe in newPoints:
                t_pb=pb
                t_pe=pe
                sb = t_pb.split('_')
                se = t_pe.split('_')
                if sb[0] != se[0] and newPoints[pb][3] == 'Active' and newPoints[pe][3] == 'Active':
                    dis = getDistance(newPoints[pb][0],newPoints[pe][0],newPoints[pb][1],newPoints[pe][1])*1000
                    if dis < e and dis < minDis:
                        lisDis.clear()
                        minDis = dis
                        lisDis.append(pb)
                        lisDis.append(pe)
                        lisDis.append(dis)
                        lisDis.append(newPoints[pb][0])
                        lisDis.append(newPoints[pe][0])
                        lisDis.append(newPoints[pb][1])
                        lisDis.append(newPoints[pb][1])
                        noMore = True
        if noMore == True:
            newLat = (lisDis[3] + lisDis[5])/2
            newLon = (lisDis[4] + lisDis[6])/2
            pointLocation = [newLat,newLon,'N','Active',{},0]
            newPoints[lisDis[0]][3] = 'Merged with '+lisDis[1]+' '
            newPoints[lisDis[1]][3] = 'Merged with '+lisDis[0]+' '
        else:
            return    


mainStops = {
            'A':['Областная больница','Regional Hospital',49.462464,32.011329],
            'B':['Генерала Момота','General Momot',49.454207,32.018796],
            'C':['Ул. Пацаева','st. Patsaeva',49.405708,32.101672],
            'D':['Ул. Сумгаитская','st. Sumgayit',49.406165,32.115508],
            'E':['Грузпорт','Cargo port',49.430336,32.1014207],
            'F':['Азот','Azot',49.365936,32.070388],
            'G':['Центр','Center',49.442511,32.062038],
            'H':['Аврора','Aurora',49.480718,32.000622],
            'J':['Речной вокзал','River Station',49.439763,32.095566],
            'K':['Автовокзал','Bus station',49.406505783183476, 32.01664375414617],
            'L':['ЖД вокзал','Railway station',49.42603112858668, 32.04980129681282],
            'M':['Ул. Рустави','st. Rustavi',49.440220,32.003355],
            'N':['Дахновка','Dakhnovka',49.486592,31.970025],
            'O':['Ул. Сурикова','st. SurikDakhnovkaov',49.38922941801914, 32.036848],
            'P':['Госпиталь','Hospital',49.406157,32.115501],
            'Q':['Мытница','Mytnitsa',49.443837,32.077811],
            'R':['Ул. Нарбутовская','st. Narbutovskaya',49.419250,32.084867]
            }

lines = {'4':['A','C'],'5':['B','H'],'6':['A','K'],'7':['B','E'],'8':['G','O'],'11':['H','P'],
         '20':['M','J'],'21':['J','C'],'22':['H','D'],'24':['C','K'],'25':['H','D'],'26':['A','K'],
         '27':['L','R'],'28':['C','K'],'29':['M','Q'],'31':['N','F'],'36':['J','D']
        }

stationfile = open("Names_and_distance.txt",'w')
#print("d1- Distance from previous point")
#print("d2- Distance from begining point")

newPoints = {}
e = 200
countAll = 0
for m in lines:
    lFrom = lines[m][0]
    lTo = lines[m][1]
    if lFrom != '' and lTo != '':
        latf = mainStops[lFrom][2]
        latt = mainStops[lTo][2]
        lonf = mainStops[lFrom][3]
        lont = mainStops[lTo][3]
        d = getDistance(latf,latt,lonf,lont)
        txttoWrite = "{:s}:{:s}-{:s}, d={:.2f} km {:.10f},{:.10f} \n" .format(
            m,mainStops[lFrom][1],mainStops[lTo][1],d,mainStops[lFrom][2],mainStops[lFrom][3])
        stationfile.write(txttoWrite)
        n = int((d*1000)/e)
        latstep =  (latt - latf)/n
        lonstep =  (lont - lonf)/n
        countLine = 0
        for x in range(1,n):
            pointName = m+"_"+str(x)
            new_lat = latf + latstep
            new_lon = lonf + lonstep
            pointLocation = [new_lat,new_lon,m,'Active',{},0]
            newPoints[pointName] = pointLocation
            txttoWrite = "{:s} {:.10f},{:.10f}\n".format(pointName,new_lat,new_lon)
            stationfile.write(txttoWrite)
            latf = new_lat
            lonf = new_lon
            countLine+=1
            countAll+=1
    txttoWrite = "{:s}:{:s}-{:s}, d={:.2f} km {:d} points, {:.10f},{:.10f} \n\n".format(
        m,mainStops[lFrom][1],mainStops[lTo][1],d,countLine,mainStops[lTo][2],mainStops[lTo][3])
    stationfile.write(txttoWrite)

txttoWrite = "Number of points={:d}\n".format(countAll)
stationfile.write(txttoWrite)
stationfile.close()

calcNearStations(newPoints,e)    
printFinalList('BeforeSimilar',newPoints,'Active')
calcSimilarStations(newPoints,e)
printFinalList('AfterSimilar',newPoints,'ALL')
calcMin(newPoints,e)
printFinalList('AfterMergin',newPoints,'ALL')
print(datetime.datetime.now())        

0
['11_50', '21_13', 6.791659392067806, 49.416441275861935, 49.41646221052635, 32.09965562068968, 32.09965562068968]
1
['4_16', '29_14', 12.958997035237047, 49.44228408888891, 49.44216761538458, 32.04345095555557, 32.04345095555557]
2
['7_16', '11_30', 14.131023516796512, 49.44227149999995, 49.44215196551716, 32.06010835, 32.06010835]
3
['20_1', '29_1', 17.739505988002158, 49.44020615151515, 49.44035911538461, 32.00614927272727, 32.00614927272727]
4
['8_21', '24_20', 19.727963703867765, 49.40641702510973, 49.406239855455674, 32.04497380645164, 32.04497380645164]
5
['7_19', '20_23', 22.25782758016847, 49.44003359374994, 49.4399014848485, 32.067854415625, 32.067854415625]
6
['4_9', '7_4', 23.275429687962937, 49.45111280000001, 49.451223124999984, 32.02939760000001, 32.02939760000001]
7
['7_15', '29_19', 24.347614027783987, 49.44301746874995, 49.44286319230764, 32.057526328125, 32.057526328125]
8
['20_2', '29_2', 35.47900834595443, 49.4401923030303, 49.44049823076922, 32.00894354545454, 3

['4_35', '27_12', 166.6698495286162, 49.4183204444445, 49.419771625275864, 32.08159577777781, 32.08159577777781]
71
['7_1529_194_3427_114_3327_107_1429_188_220_204_3227_9', '4_3527_12', 124.23360960972693, 40.74972482602377, 40.749958111111155, 40.74924188190572, 40.74924188190572]
72
['11_56', '36_18', 168.12561044346523, 49.40872806896537, 49.409524800000014, 32.11153965517244, 32.11153965517244]
73
['4_24', '8_6', 169.95290736659027, 49.43219413333337, 49.43219843574564, 32.059511933333354, 32.059511933333354]
74
['8_2124_2020_329_3', '20_229_26_1631_336_1531_32', 173.26674978590606, 40.72565112336717, 40.72445922994426, 40.72578248365864, 40.72578248365864]
75
['21_236_2', '7_3121_521_336_3', 175.16630347965298, 40.7661935, 40.76489012072366, 40.76630596842105, 40.76630596842105]
76
['6_18', '31_35', 176.86690883788222, 49.42997213217102, 49.43102673684221, 32.014414986278425, 32.014414986278425]
77
['8_2431_5424_2331_51', '6_1831_35', 107.98359956259489, 40.72179890221428, 40.7221

4
